In [ ]:
%cd /
from google.colab import drive
drive.mount('/content/gdrive/')

!ln -s /content/gdrive/My\ Drive/ ./mydrive
%cd /content/gdrive/MyDrive/projekt_inzynierski
!ls

# Data import

## Dataset

Dane pochodzą z: https://www.kaggle.com/datasets/undefinenull/million-song-dataset-spotify-lastfm. Zostały pobrane z Last.fm oraz Spotify API. Zawierają tagi dodane przez użytkowników Last.fm oraz dane liczbowe na temat utworów.


---



**track_id** - ID z datasetu

**name** - Nazwa piosenki

**artist** - Wykonawca

**spotify_preview_url** - URL fragmentu utworu w Spotify

**spotify_id** - ID utworu w serwisie Spotify

**tags** - Lista tagów z Last.fm

**genre** - Gatunek

**year** - Rok, w którym został wydany utwór

**duration_ms** - Czas trwania w ms

**danceability** - Opisuje jak bardzo utwór nadaje się do tańca na podstawie m.in. tempa, rytmu. 0 - nienadający się tańca, 1 - taneczny utwór.

**energy** - Energia, intensywność, aktywność utworu. Związana z cechami takimi jak prędkość, głośność.

**key** - Klucz utworu, liczba całkowita. Mapowanie liczba - klucz jest na podstawie "Pitch Class notation". Gdy nie wykryto żadnego klucza wartość to -1.

**loudness** - Głośćność w dB. Średnia głośność na przestrzeni całej długości utworu. Zazwyczaj pomiędzy -60 a 0 dB.

**mode** - Skala: major (1) lub minor (0).

**speechiness** - Procent słow mówionych w piosence. Bliżej 1 są podcasty, a wartości pomiędzy 0,33 a 0,66 to najczęściej utwory zawierające mowę, np. rap.

**acousticness** - Pewność, z jaką utwór jest akustyczny (1 - wysokie prawdopodobieństwo).

**instrumentalness** - Intrumentalność. Im bliżej 1, tym większa pewność, że utwór nie zawiera słów śpiewanych lub rapowanych.

**liveness** - Pewność, z jaką utwór był wykonywany na żywo. Wysokie wartości znaczą, że w nagraniu zostały wykryte odgłosy widowni.

**valence** - Pozytywność. Wysokie wartości (bliżej 1) oznaczają pozytywne utwory, np radosne, euforyczne. Natomiast niskie valence oznacza, że utwór jest negatywny (np. dominuje emocja smutku, złości).

**tempo** - Tempo utworu w BPM.

**time_signature** - Metrum. Określa, liczbę uderzeń w jednym takcie.
Parametr ten przyjmuje wartości od 3 do 7, co odpowiada metrum w zakresie od „3/4” do „7/4”.

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("df_kaggle_enriched_cleaned.csv")
df.sample(5)

In [ ]:
columns = df.columns.tolist()
columns

Normalizacja danych - dane takie jak tempo, loudness normalizujemy do skali 0-1. Pozostałe dane są już w takiej skali.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()
df["n_tempo"] = scaler.fit_transform(df[["tempo"]])
df["n_loudness"] = scaler.fit_transform(df[["loudness"]])

In [ ]:
df[["name", "artist", "tags", "genre", "tempo", "loudness", "n_tempo", "n_loudness"]].sample(5)

Czyszczenie danych

In [ ]:
df.info()

Usunięcie NaN

In [ ]:
((df.isna().sum() / len(df)) * 100).round()

In [ ]:
df[df["spotify_id"].isna()]

In [ ]:
df["tags"] = df["tags"].fillna("")
df["genre"] = df["genre"].fillna("")

Tworzymy listę tagów - na potrzeby bazy danych, oraz do dalszej analizy

In [ ]:
df["tags_list"] = df["tags"].apply(lambda x: [t for t in x.split(", ") if t != ''])

df[["name", "artist", "tags", "tags_list"]].sample(5)

Sprawdzenie duplikatów

In [ ]:
df[df.duplicated(subset=['spotify_id'], keep=False)]

In [ ]:
df['tags_count'] = df['tags_list'].apply(len)

In [ ]:
df['orig_index'] = df.index

df_sorted = df.sort_values(['spotify_id', 'tags_count'], ascending=[True, False])

# drop dupliactes, leave the one with biggest number of tags
df_unique = df_sorted.drop_duplicates(subset='spotify_id', keep='first').reset_index(drop=True)
df = df_unique.sort_values('orig_index').drop(columns='orig_index')

df[df.duplicated(subset=['spotify_id'], keep=False)]

In [ ]:
df[df.duplicated(subset=["name", "artist"])]

Sprawdzamy, czy tagi są czyste, w formie małych liter, wystarczająco unikalne (bez zbędnych powtórzeń).

In [ ]:
all_tags = df["tags_list"].explode()
tag_counts = all_tags.value_counts()
tag_counts


In [ ]:
tags = tag_counts.index.tolist()
tags

In [ ]:
df.to_csv("df_kaggle_enriched_cleaned.csv", index=False)

# Analiza danych

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df.select_dtypes(include='object').describe()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# cols_to_exclude = ['n_loudness', 'n_tempo']
cols_to_exclude = []

In [ ]:
numeric_data = df.select_dtypes(include=['float64', 'int64']).columns

numeric_data = [col for col in numeric_data if col not in cols_to_exclude]

df[numeric_data].hist(bins='auto', figsize=(15, 10))
plt.suptitle("Histogramy danych numerycznych", fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
numeric_df = df.select_dtypes(include='number')

plt.figure(figsize=(12, 10))

numeric_df = numeric_df.drop(columns=cols_to_exclude, errors='ignore')

sns.heatmap(numeric_df.corr(), annot=True, fmt='.2f', square=True)
plt.title('Macierz korelacji', fontsize=16)
plt.show()

# Embeddings

Import biblioteki z modelami do embeddingu SBERT

In [ ]:
from sentence_transformers import SentenceTransformer, util

Wybór odpowiedniego modelu

In [ ]:
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

In [ ]:
tags = "rock, alternative, 90s, funk"

contexts = [
    "muzyka do pracy",
    "muzyka do tańca",
    "muzyka do nauki",
    "muzyka na siłownię",
    "muzyka do relaksu"
]

emb_tags = model.encode(tags, convert_to_tensor=True)
emb_contexts = model.encode(contexts, convert_to_tensor=True)

scores = util.cos_sim(emb_tags, emb_contexts)
for i, ctx in enumerate(contexts):
    print(f"{ctx}: {scores[0][i]:.3f}")

In [ ]:
tags = "rock"

contexts = [
    "muzyka do pracy",
    "muzyka do tańca",
    "muzyka do nauki",
    "muzyka na siłownię",
    "muzyka do relaksu",
    "muzyka do gotowania",
]

emb_tags = model.encode(tags, convert_to_tensor=True)
emb_contexts = model.encode(contexts, convert_to_tensor=True)

scores = util.cos_sim(emb_tags, emb_contexts)
for i, ctx in enumerate(contexts):
    print(f"{ctx}: {scores[0][i]:.3f}")